In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import LSTM, Convolution1D, Flatten, Dropout,MaxPooling1D
# from keras.layers.convolution import 
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.callbacks import TensorBoard
import ast
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
import keras as kr
from keras.callbacks import ModelCheckpoint
kr.__version__

Using TensorFlow backend.


'2.0.4'

In [3]:
base_dir_tr='training/'
base_dir_vl='validation/'
weight_log='weight_logs/'

In [66]:
top_words = 20000
dtr_1=pd.read_csv(base_dir_tr+'data_manipulation_2_2_train.csv',encoding='ISO-8859-1')
dtv_1=pd.read_csv(base_dir_vl+'data_manipulation_2_2_valid.csv',encoding='ISO-8859-1')

In [67]:
Xsd=dtr_1.rank_short_des.as_matrix()
Xsd=np.array([ast.literal_eval(cd) for cd in Xsd])

Xti=dtr_1.rank_title.as_matrix()
Xti=np.array([ast.literal_eval(cd) for cd in Xti])

yco=dtr_1.conci

In [68]:
Xsd_v=dtv_1.rank_short_des.as_matrix()
Xsd_v=np.array([ast.literal_eval(cd) for cd in Xsd_v])

Xti_v=dtv_1.rank_title.as_matrix()
Xti_v=np.array([ast.literal_eval(cd) for cd in Xti_v])

In [69]:
max_review_length_s = 250
max_review_length_t = 60
Xsd = sequence.pad_sequences(Xsd, maxlen=max_review_length_s)
Xti = sequence.pad_sequences(Xti, maxlen=max_review_length_t)

Xsd_v = sequence.pad_sequences(Xsd_v, maxlen=max_review_length_s)
Xti_v = sequence.pad_sequences(Xti_v, maxlen=max_review_length_t)

In [70]:
X=[]
for i,j in zip(Xsd,Xti):
    X.append(list(j)+list(i))
X=np.array(X)

X_v=[]
for i,j in zip(Xsd_v,Xti_v):
    X_v.append(list(j)+list(i))
X_v=np.array(X_v)

# np.savetxt(base_dir_vl+'data_input_yco_3_1.txt',X_v)
# np.savetxt(base_dir_vl+'data_input_yco_3_2.txt',X_v)
# np.savetxt(base_dir_vl+'data_input_yco_3_3.txt',X_v)
# np.savetxt(base_dir_vl+'data_input_yco_3_4.txt',X_v)
np.savetxt(base_dir_vl+'data_input_yco_3_5.txt',X_v)

In [71]:
X[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 7533,
          0,  363, 1959,   42,   37,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

#### yco model

In [72]:
# model.save('model_yco_1_2_3.h5') OKE (74.05%)
yco.value_counts()/yco.value_counts().sum()

1    0.685335
0    0.314665
Name: conci, dtype: float64

In [73]:
X_train, X_test, yco_train, yco_test = train_test_split(X, yco, test_size=0.3, random_state=21)

In [74]:
len(X_train)

25398

In [ ]:
#### Creating the model_1

np.random.seed(21)

# Using embedding from Keras
embedding_vecor_length = 150
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length_s+max_review_length_t))

# model.add(Convolution1D(256, 3, border_mode='same'))
# model.add(Convolution1D(128, 3, border_mode='same'))
model.add(Convolution1D(128, 3, border_mode='same'))
model.add(Convolution1D(32, 3, border_mode='same'))
# model.add(Convolution1D(16, 3, border_mode='same'))
# model.add(Convolution1D(8, 3, border_mode='same'))
# model.add(LSTM(150))
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(500,activation='sigmoid'))
# model.add(Dense(500,activation='sigmoid'))
# model.add(Dense(150,activation='sigmoid'))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))

model2=model

# Log to tensorboard
# tensorBoardCallback = TensorBoard(log_dir='./logs', write_graph=True)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy','mse'])

# checkpoint
# filepath=weight_log+"weights_best_yco_3_1.hdf5"
# filepath=weight_log+"weights_best_yco_3_2.hdf5"
# filepath=weight_log+"weights_best_yco_3_3.hdf5"
# filepath=weight_log+"weights_best_yco_3_4.hdf5"
filepath=weight_log+"weights_best_yco_3_5.hdf5"

checkpoint = ModelCheckpoint(filepath, monitor='val_mean_squared_error', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]


C:\Users\Aska\Anaconda2\envs\snakes3_gpu\lib\site-packages\ipykernel\__main__.py:12: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(128, 3, padding="same")`
C:\Users\Aska\Anaconda2\envs\snakes3_gpu\lib\site-packages\ipykernel\__main__.py:13: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(32, 3, padding="same")`


In [ ]:
model.fit(X_train, yco_train, validation_data=(X_test,yco_test),callbacks=callbacks_list, epochs=20, batch_size=100, verbose=1)

Train on 25398 samples, validate on 10885 samples
Epoch 1/20
22200/25398 [=========================>....] - ETA: 61s - loss: 0.4688 - acc: 0.7876 - mean_squared_error: 0.1509

In [14]:
# model2.load_weights(weight_log+"weights_best_yco_3_1.hdf5")
# model2.load_weights(weight_log+"weights_best_yco_3_2.hdf5")
# model2.load_weights(weight_log+"weights_best_yco_3_3.hdf5")
# model2.load_weights(weight_log+"weights_best_yco_3_4.hdf5")
model2.load_weights(weight_log+"weights_best_yco_3_5.hdf5")

model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy','mse'])

In [15]:
# Evaluation on the test set
yco_predc=model2.predict_classes(X_test)
yco_predp=model2.predict(X_test)

yco_predc=[i[0] for i in yco_predc]
yco_predp=[i[0] for i in yco_predp]

scores = model2.evaluate(X_test, yco_test, verbose=0)
mse_c=mse(yco_test,yco_predc)
mse_p=mse(yco_test,yco_predp)
print("Accuracy: %.2f%%" % (scores[1]*100)) 
print("MSE_class : ",mse_c)
print("MSE_prob : ",mse_p)

10885/10885 [==============================] - 85s    
Accuracy: 82.28%
MSE_class :  0.177216352779
MSE_prob :  0.130504133577


In [16]:
# model.save('model_yco_2_2.h5') # MSE_prob 82.75%  0.128415694158
# model2.save('model_yco_3_1.h5') # MSE 81.81% 0.13448
# model2.save('model_yco_3_2.h5') # MSE_prob 82.11% 0.134357
# model2.save('model_yco_3_3.h5') # MSE prob 82.09% 0.1310365
# model2.save('model_yco_3_4.h5') # 82.35%     0.13095322
model2.save('model_yco_3_5.h5')

In [ ]:
# model.save('model_yco_3_2.h5') # MSE_prob 82.11% 0.134357

np.random.seed(21)

# Using embedding from Keras
embedding_vecor_length = 150
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length_s+max_review_length_t))

# model.add(Convolution1D(256, 3, border_mode='same'))
model.add(Convolution1D(128, 3, border_mode='same'))
model.add(Convolution1D(32, 3, border_mode='same'))
model.add(Convolution1D(16, 3, border_mode='same'))
model.add(Convolution1D(8, 3, border_mode='same'))
# model.add(LSTM(150))
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(150,activation='sigmoid'))
model.add(Dense(150,activation='sigmoid'))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))

model2=model

# Log to tensorboard
# tensorBoardCallback = TensorBoard(log_dir='./logs', write_graph=True)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy','mse'])


In [28]:
# model.save('model_yco_2_2.h5') <- 2 epoch,. 100 each
# Using embedding from Keras
embedding_vecor_length = 150
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length*2))

# Convolutional model (3x conv, flatten, 2x dense)
model.add(Convolution1D(256, 3, border_mode='same'))
# model.add(Convolution1D(128, 3, border_mode='same'))
model.add(Convolution1D(64, 3, border_mode='same'))
model.add(Convolution1D(32, 3, border_mode='same'))
model.add(Convolution1D(16, 3, border_mode='same'))
model.add(Convolution1D(8, 3, border_mode='same'))
# model.add(Convolution1D(4, 3, border_mode='same'))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(180,activation='sigmoid'))
# model.add(Dense(130,activation='sigmoid'))
# model.add(Dense(130,activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))

# Log to tensorboard
tensorBoardCallback = TensorBoard(log_dir='./logs', write_graph=True)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy','mse'])

### Validation

In [4]:
# X_vl=np.loadtxt(base_dir_vl+'data_input_yco_3_2.txt')
# yco_model=load_model('model_yco_3_2.h5')

# X_vl=np.loadtxt(base_dir_vl+'data_input_yco_3_3.txt')
# yco_model=load_model('model_yco_3_3.h5')

X_vl=np.loadtxt(base_dir_vl+'data_input_yco_3_4.txt')
yco_model=load_model('model_yco_3_4.h5')

# X_vl=np.loadtxt(base_dir_vl+'data_input_yco_3_5.txt')
# yco_model=load_model('model_yco_3_5.h5')


In [7]:
yco_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 200, 150)          3000000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 200, 128)          57728     
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 200, 32)           12320     
_________________________________________________________________
flatten_1 (Flatten)          (None, 6400)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 6400)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 500)               3200500   
_________________________________________________________________
dense_2 (Dense)              (None, 500)               250500    
__________

In [18]:
yco_predp=yco_model.predict(X_vl)
yco_predp=[i[0] for i in yco_predp]

In [19]:
with open('conciseness_valid.predict','w') as fco:
    for pp in yco_predp:
        fco.write(str(pp))
        fco.write('\n')
fco.close()